In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install transformers trl wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [3]:
!wandb login

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: techitotamani (techitotamani-tech_it-o) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Data load

In [4]:
import json
from datasets import Dataset, DatasetDict

with open("Data_RandomDate_finetune-Tell.json", "r", encoding="utf-8") as f:
    data_dict = json.load(f)


dataset_dict = DatasetDict({
    "train": Dataset.from_list(data_dict['train']),
    "valid": Dataset.from_list(data_dict['valid']),
    "test": Dataset.from_list(data_dict['test']),
})

In [5]:
with open("tools.json", "r", encoding="utf-8") as f:
    tools = json.load(f)

In [6]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2040
    })
    valid: Dataset({
        features: ['messages'],
        num_rows: 584
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 296
    })
})

# **Train**

In [7]:
import random
import numpy as np
import torch


SEED = 3407


random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [8]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


max_seq_length = 2048
dtype = None
load_in_4bit = False
MODEL_NAME = "unsloth/Qwen3-0.6B"
SAVE_DIR = "lora_model"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [9]:
# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    use_rslora=False,
    loftq_config=None,
)


def formatting_prompts_func(data):

    texts = []

    for messages in data["messages"]:

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            tools=tools,
            enable_thinking=False
        )
        texts.append(text)
    return {"text": texts}

Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
train_dataset = dataset_dict["train"]
valid_dataset = dataset_dict["valid"]

In [12]:
train_dataset

Dataset({
    features: ['messages'],
    num_rows: 2040
})

In [13]:
train_dataset[1]

{'messages': [{'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-02-01.\n\nCurrent Day: Saturday.',
   'role': 'system'},
  {'content': 'เอานัดทำการบ้านกับเพื่อนพฤหัสที่จะถึงนี้ออก', 'role': 'user'},
  {'content': "<tool_call>\n{'name': 'delete_event_date', 'arguments': {'date': '2025-02-06', 'title': 'ทำการบ้านกับเพื่อน'}}\n</tool_call>",
   'role': 'assistant'}]}

In [14]:
train_dataset = train_dataset.map(formatting_prompts_func, batched=True, num_proc=2)
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True, num_proc=2)

Map (num_proc=2):   0%|          | 0/2040 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/584 [00:00<?, ? examples/s]

In [15]:
train_dataset[1]

{'messages': [{'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-02-01.\n\nCurrent Day: Saturday.',
   'role': 'system'},
  {'content': 'เอานัดทำการบ้านกับเพื่อนพฤหัสที่จะถึงนี้ออก', 'role': 'user'},
  {'content': "<tool_call>\n{'name': 'delete_event_date', 'arguments': {'date': '2025-02-06', 'title': 'ทำการบ้านกับเพื่อน'}}\n</tool_call>",
   'role': 'assistant'}],
 'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-02-01.\n\nCurrent Day: Saturday.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "add_event_date", "description": "Add event by title and date.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of event ."}, "date": {"type": "string", "descri

In [16]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        eval_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        save_total_limit=2,
        load_best_model_at_end=True,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=SEED,
        output_dir="Qwer3-0.6B-AddDayTelling-2Epoc",
        report_to="wandb",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/2040 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/584 [00:00<?, ? examples/s]

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,040 | Num Epochs = 2 | Total steps = 510
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544/606,142,464 (1.67% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: techitotamani (techitotamani-tech_it-o) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.067200,0.062797
100,0.041900,0.041713
150,0.042300,0.037478


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
50,0.067200,0.062797
100,0.041900,0.041713
150,0.042300,0.037478
200,0.031300,0.035321
250,0.037400,0.034813
300,0.030100,0.033945
350,0.031900,0.033012
400,0.033500,0.032737
450,0.033200,0.032293
500,0.032300,0.032014


TrainOutput(global_step=510, training_loss=0.06565335013118445, metrics={'train_runtime': 1901.3081, 'train_samples_per_second': 2.146, 'train_steps_per_second': 0.268, 'total_flos': 8382259691520000.0, 'train_loss': 0.06565335013118445})

In [18]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\n Current Date: 2025-05-06"},
    {"role": "user", "content": "เอานัดคุยงานวันที่ 14 ธันวา ออก	"},
]

In [19]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False,
    tools=tools,
    enable_thinking=False
)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
output_text = tokenizer.batch_decode(outputs)[0][len(text):]

In [20]:
output_text

"<|im_start|>assistant\n<think>\n\n</think>\n\n<tool_call>\n{'name': 'delete_event_date', 'arguments': {'date': '2025-12-14', 'title': 'คุยงาน'}}\n</tool_call><|im_end|>"

# **Push**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Qwen_finetune_9-4B` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

In [ ]:
model.push_to_hub("TechitoTamani/Qwen3-0.6B_FinetuneWithMyData", private=True)
tokenizer.push_to_hub("TechitoTamani/Qwen3-0.6B_FinetuneWithMyData", private=True)

README.md:   0%|          | 0.00/558 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/132M [00:00<?, ?B/s]

Saved model to https://huggingface.co/TechitoTamani/Qwen3-4B_FinetuneWithMyData-3-AddDayTelling-2


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear